# 01 - Data Cleaning & Exploration

This notebook loads, cleans, and explores the alternative dataset for Tata Motors analysis.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from src.data_cleaning import load_data, clean_data, save_cleaned_data

In [ ]:
# Load data
raw_df = load_data('data/simulated_auto_sales.csv')
raw_df.head()

In [ ]:
# Clean data
cleaned_df = clean_data(raw_df)
cleaned_df.head()

In [ ]:
# Save cleaned data
save_cleaned_data(cleaned_df, 'data/cleaned_auto_sales.csv')

In [ ]:
# Basic EDA
cleaned_df.describe()

In [ ]:
cleaned_df.isnull().sum()

In [ ]:
cleaned_df.plot(figsize=(12,6), title='Cleaned Auto Sales Data')
plt.tight_layout()
plt.show()